In [5]:
#encoding=utf-8
import pandas as pd
import numpy as np
import re
import string

In [6]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../../data/IDClassifier/'
data = pd.read_csv(path + 'mock_up_data.csv', encoding='utf8')
data = data.drop(['id','text','备注'], axis = 1)
data = data.dropna()
data.head(10)

,split_text,label
0,啊,1.0
1,你 有病 吧 我 跟你 说过 很多次 了 我 不是 他,2.0
2,不是 什么事,2.0
3,啊 对,1.0
5,对,1.0
6,对,1.0
8,我 不是 啊,2.0
9,噢,1.0
10,啊,1.0
12,啊,1.0


# use goslate to get more data

In [7]:
from textblob import TextBlob
from textblob.translate import NotTranslated
import pandas as pd
import numpy as np
from multiprocessing import Pool
from itertools import repeat

In [8]:
def translate(comment, from_lang, to_lang):
        if hasattr(comment, "decode"):
            comment = comment.decode("utf-8")

        text = TextBlob(comment)
        try:
            text = text.translate(to=to_lang)
            text = text.translate(to=from_lang)
        except NotTranslated:
            pass

        return str(text)

def translate_csv(df,col,from_lang,to_lang,num_pol=10):
    """
        https://developers.google.com/translate/v2/using_rest#language-params
    """

    df = df.copy()

    if num_pol >= 1000:
        num_pol=1000
    comment_pool = df[col].values
    p = Pool(num_pol)
    new_col_name = col + '_' + to_lang
    df[new_col_name] = p.starmap(translate, zip(comment_pool, repeat(from_lang),repeat(to_lang)))
    df = df.drop([col], axis = 1)
    df = df.rename(index=str, columns={new_col_name:col})
    return df

In [9]:
col = 'split_text'
data_en = translate_csv(data,col,from_lang='zh',to_lang='en',num_pol=50)
print('finish 1st trans')
data_fr = translate_csv(data,col,from_lang='zh',to_lang='fr',num_pol=50)
print('finish 2nd trans')
data_th = translate_csv(data,col,from_lang='zh',to_lang='th',num_pol=50)
print('finish 3rd trans')
data_lo = translate_csv(data,col,from_lang='zh',to_lang='lo',num_pol=50)
print('finish 4th trans')

In [17]:
data = pd.concat([data,data_en,data_fr,data_th,data_lo], ignore_index=True)

In [18]:
import jieba
jieba.load_userdict("../../WordCut/userdict.txt")
def cut_words(text):
    seg_list = jieba.cut(text, cut_all=False)
    return " ".join(seg_list)

data['split_text']=data['split_text'].apply(cut_words)

# data cleaning and save

In [13]:
def clean(text):
    text = re.sub(f'([{string.punctuation}“”¨«»®´·º ½¾¿¡§£₤‘’，])',' ', text)
    text = text.split(' ')
    text = ' '.join(text)
    return text

def clean_label(label):
    label = int(label)
    a = np.zeros(3)
    a[label-1] = 1
    return a.astype(int).tolist()

def clean_label(label):
    return int(label-1)
    
print('split_text cleaning')
# data['split_text'].head(10)
data['split_text'] = data['split_text'].apply(clean)
data['label'] = data['label'].apply(clean_label)

data.to_csv(path + 'cleaned_mock_up_data.csv', index = False, encoding = 'utf8')

split_text cleaning


In [14]:
data.head(10)

,label,split_text
0,0,啊
1,1,你 有病 吧 我 跟 你 说 过 很 多次 了 我 不是...
2,1,不是 什么 事
3,0,啊 对
4,0,对
5,0,对
6,1,我 不是 啊
7,0,噢
8,0,啊
9,0,啊


In [15]:
data.label.values

array([0, 1, 1, ..., 1, 2, 2])

In [16]:
data[data['label']==2]

,label,split_text
68,2,你 说 什么 我 没 听 清
69,2,什么
70,2,你 打 错 电话 了
71,2,不要 再 打 这个 电话 了 烦 不 烦
72,2,你好
73,2,唉 你好
74,2,你 找 哪位
79,2,我 现在 不 方便 你 一会儿 再 打 来
80,2,什么 事
81,2,你 再说 一遍 我 没 听 清
